In [1]:
import multiprocessing
import os
import time
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d
import util
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import random

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
kdtree = None
radius = 1.3

def init_kdtree(tree):
    global kdtree
    kdtree = tree

def find_plane_directions(indices, points, radius=2):
    normals = []
    for idx in indices:
        neighbor_indices = kdtree.query_ball_point(points[idx], radius)
        neighbors = points[neighbor_indices]
        mean = np.mean(neighbors, axis=0)
        norm = neighbors - mean
        cov = np.cov(norm.T)
        eig_val, eig_vec = np.linalg.eigh(cov)
        sorted_idx = np.argsort(eig_val)[::-1]
        eig_vec = eig_vec[:, sorted_idx]
        plane_direction = eig_vec[:, 2]
        normals.append(plane_direction)
    return normals

def calculate_normal_standard_deviation(indices, points, normals, radius=2):
    standard_deviations = []

    for idx in indices:
        # **Step 1: Neighbor Query**
        neighbor_indices = kdtree.query_ball_point(points[idx], radius)

        # **Step 2: Retrieve Neighbor Normals (Optimized)**
        neighbor_normals = normals[neighbor_indices]  # Direct indexing (no conversion)

        if len(neighbor_normals) == 0:
            standard_deviations.append(0)  
            continue

        # **Step 3: Align Normals**
        reference_normal = neighbor_normals[0]
        dot_products = np.dot(neighbor_normals, reference_normal)
        aligned_normals = neighbor_normals * np.sign(dot_products)[:, np.newaxis]

        # **Step 4: Standard Deviation Calculation**
        std_dev = np.std(aligned_normals, axis=0)
        variation_measure = np.sum(std_dev)  

        standard_deviations.append(variation_measure)

    return standard_deviations  

if __name__ == "__main__": 
    dataname = "/home/chris/Code/PointClouds/data/ply/CircularVentilationGrateExtraCleanedFull.ply"
    pcd = o3d.io.read_point_cloud(dataname)
    pcd = util.preProcessCloud(pcd)
    myarray = np.asarray(pcd.points)
    indices = np.arange(len(myarray))
    kdtree = cKDTree(myarray)
    num_chunks = 16
    chunk_size = len(myarray) // num_chunks
    chunked_indices = [indices[i:i + chunk_size] for i in range(0, len(indices), chunk_size)]

    # First pass: Compute normals
    start_time_first_pca = time.time()
    with multiprocessing.Pool(processes=4, initializer=init_kdtree, initargs=(kdtree,)) as pool:
        normals_chunks = pool.starmap(find_plane_directions, [(chunk_indices, myarray, radius) for chunk_indices in chunked_indices])
    first_pca_duration = time.time() - start_time_first_pca
    print(f"First PCA time: {first_pca_duration:.2f} seconds")

    # Flatten normals to a single array in order
    all_normals = np.vstack(normals_chunks)

    # # Flatten standard deviations to maintain order
    # standard_deviations = np.hstack(standard_deviation_chunks)

    # # Normalize variation values
    # max_variation = np.max(standard_deviations) if len(standard_deviations) > 0 else 1
    # standard_deviations /= max_variation

    # Single-core execution check
    start_time_single_core = time.time()

    # Pick the first chunk for testing
    test_chunk = chunked_indices[0]

    # Run without multiprocessing
    results_single = calculate_normal_standard_deviation(test_chunk, myarray, all_normals, radius)

    # Print timing results
    single_core_duration = time.time() - start_time_single_core
    print(f"Single-core execution time: {single_core_duration:.2f} seconds")

    # Print first 5 results to verify
    print("First 5 standard deviations (single-core):")
    print(results_single[:5])

First PCA time: 132.81 seconds
Total iterations: 132141
Average query time: 0.000036 sec, Total: 4.73 sec
Average retrieval time: 0.000019 sec, Total: 2.46 sec
Average alignment time: 0.000014 sec, Total: 1.81 sec
Average std calculation time: 0.000066 sec, Total: 8.72 sec
Total function execution time: 17.73 sec
Single-core execution time: 18.22 seconds
First 5 standard deviations (single-core):
[0.01365144150492793, 0.014245041924701855, 0.014844887086097758, 0.0153503349845382, 0.015227066100826067]


In [3]:
test_chunk = chunked_indices[0]
print(f"Number of points in test chunk: {len(test_chunk)}")  # Add this line

Number of points in test chunk: 132141
